In [38]:

import json
import streamlit
import pandas as pd
import requests
import psycopg2 
import plotly.express as px
import plotly.graph_objects as go


In [39]:
#create dataframe from sql
#sql connection
mydb = psycopg2.connect(host = "localhost",
                        user = "postgres",
                        password = "Vignesh213@",
                        database = "phonepay_data",
                        port = 5432)
cursor = mydb.cursor()


In [40]:
# Aggregated transaction
cursor.execute(" select * from aggregated_transaction;")
mydb.commit()
table1 = cursor.fetchall()
Aggre_trans = pd.DataFrame(table1,columns=("States","Years","Quarter","Transaction_type","Transaction_count","Transaction_amount"))

In [41]:
# Aggregated user
cursor.execute("select * from aggregated_user;")
mydb.commit()
table2 = cursor.fetchall()
Aggre_user = pd.DataFrame(table2,columns=("States","Years","Quarter","Brand","Transaction_count","Percentage"))

In [42]:
# map transaction
cursor.execute("select * from map_transaction;")
mydb.commit()
table3 = cursor.fetchall()
Map_trans = pd.DataFrame(table3,columns=("States","Years","Quarter","Transaction_name","Transaction_count","Transaction_count"))



In [43]:
# map user 
cursor.execute("select * from map_user;")
mydb.commit()
table4 = cursor.fetchall()
Map_user = pd.DataFrame(table4,columns=("States","Years","Quarter","Districts","Registeredusers","Appopens"))

In [44]:
# top transaction
cursor.execute("select * from top_transaction;")
mydb.commit()
table5 = cursor.fetchall()
Top_trans = pd.DataFrame(table5,columns=("States","Years","Quarter","Pincodes","Transaction_count","Transaction_amount"))

In [45]:
# top user
cursor.execute("select * from top_user;")
mydb.commit()
table6 = cursor.fetchall()
Top_user = pd.DataFrame(table6,columns=("States","Years","Quarter","Pincodes","Registeredusers"))

In [1]:

def animate_all_amount():

    url = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
    response = requests.get(url)
    data1 = json.loads(response.content)
    state_names_tra = [feature["properties"]["ST_NM"]for feature in data1["features"]]
    state_names_tra.sort()

    df_state_names_tra = pd.DataFrame({"States":state_names_tra})


    frames = []

    for year in Map_user["Years"].unique():
        for quarter in Aggre_trans["Quarter"].unique():

            at1 = Aggre_trans[(Aggre_trans["Years"]==year)&(Aggre_trans["Quarter"]==quarter)]
            atf1 = at1[["States","Transaction_amount"]]
            atf1 = atf1.sort_values(by="States")
            atf1["Years"]=year
            atf1["Quarter"]=quarter
            frames.append(atf1)

    merged_df = pd.concat(frames)

    fig_tra = px.choropleth(merged_df,geojson=data1,locations="States",featureidkey="properties.ST_NM",color="Transaction_amount",
                            color_continuous_scale="Sunsetdark",range_color=(0,4000000000),hover_name="States",title="TRANSACTION AMOUNT",
                            animation_frame="Years",animation_group="Quarter")

    fig_tra.update_geos(fitbounds="locations",visible = False)
    fig_tra.update_layout(width = 600,height = 700)
    fig_tra.update_layout(title_font ={"size":25})
    return st.plotly_chart(fig_tra)
